In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc
import random


In [2]:
fake = faker.Faker("ru_RU")

In [3]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 512
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 1024
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [4]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class Data(msgspec.Struct):
    data: Any

In [5]:
index = BPTree[MyKey, Data]()

In [6]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [7]:
data = []
gc.collect()

340

In [8]:
fake.random_int(0, 1000)

137

In [9]:
%%time
for i in tqdm(range(1_000_000)):
    random_int = fake.random_int(0, 99999999)
    val = f"{random_int:010} " + fake.name()
    if index.insert(MyKey(val)) is None:
        print(f"Duplicate key: {val}")
    data.append(val)

  0%|          | 0/1000000 [00:00<?, ?it/s]

CPU times: user 49.7 s, sys: 587 ms, total: 50.3 s
Wall time: 50.1 s


In [10]:
index.validate(), gc.collect()

({}, 18)

In [11]:
index.print_node(index.tree[index.root_node])


node-idx:1, prev:None, next:None, level: 1  <============ *ROOT*
   |-> MyKey(full_name='0000198910 Татьяна Захаровна Горбачева'): 0, 426
   |-> MyKey(full_name='0000393584 Дьячков Евдоким Антонович'): 426, 204
   |-> MyKey(full_name='0000579687 Галкин Моисей Филатович'): 204, 382
   |-> MyKey(full_name='0000774073 Трофимова Юлия Станиславовна'): 382, 96
   |-> MyKey(full_name='0000880950 Кулаков Эммануил Виленович'): 96, 597
   |-> MyKey(full_name='0000985511 Надежда Руслановна Тихонова'): 597, 331
   |-> MyKey(full_name='0001190478 Иларион Владиленович Агафонов'): 331, 190
   |-> MyKey(full_name='0001379379 Александрова Надежда Сергеевна'): 190, 381
   |-> MyKey(full_name='0001578346 Терентьева Елизавета Антоновна'): 381, 40
   |-> MyKey(full_name='0001775197 Феликс Филимонович Сергеев'): 40, 346
   |-> MyKey(full_name='0001978720 Евдокимов Анатолий Адамович'): 346, 158
   |-> MyKey(full_name='0002167172 Соколов Севастьян Тимурович'): 158, 341
   |-> MyKey(full_name='0002364749 Алев

In [12]:
index.min()[1]

MyKey(full_name='0000000144 Горбачева Алина Вадимовна')

In [13]:
index.max()[1]

MyKey(full_name='0099999864 Яковлева Пелагея Аскольдовна')

In [15]:
data[0:10]

['0076397250 Ангелина Ждановна Ковалева',
 '0066496171 Олимпиада Мироновна Устинова',
 '0028179657 Новиков Никита Феликсович',
 '0081528947 Агата Петровна Федотова',
 '0096843463 Феврония Борисовна Мясникова',
 '0042604684 Козлов Тит Августович',
 '0051164366 Лыткина Наталья Алексеевна',
 '0070817221 Быков Серафим Даниилович',
 '0046399124 Поляков Родион Ермолаевич',
 '0002884299 Федор Бориславович Герасимов']

In [17]:
random.shuffle(data)

In [18]:
%%time
for k in data:
        id = index.find(MyKey(k))
        if id is not None:
            pass
            # print(f"found: {id} -> {index.keys[id]}")
        else:
            index.find(k)
            print(f"Key: {k} not found")

print("Done!")

Done!
CPU times: user 10.8 s, sys: 0 ns, total: 10.8 s
Wall time: 10.8 s


In [20]:
len(data) // 11

90909